In [1]:
# pip install kaggle if you're in a non-Kaggle env
# pip install simpletransformers if you have internet

In [2]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv

worker,less_toxic,more_toxic
313," This article sucks 

woo woo wooooooo","WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=
WHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back"


In [3]:
import pandas as pd
trainer = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

toxic = trainer[["more_toxic"]]
toxic["text"] = toxic["more_toxic"]
toxic["label"] = 0.75

nontoxic = trainer[["less_toxic"]]
nontoxic["text"] = nontoxic["less_toxic"]
nontoxic["label"] = 0.35

combined = toxic.append(nontoxic).sample(frac=1, random_state=101)
combined = combined[["text", "label"]]
combined["label"] = combined["label"].astype('float')
combined.head(2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,text,label
27323,Wha's a reliable source mike? Something that c...,0.35
10783,Your contempt is noted. I hope I don't dissapp...,0.35


In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(combined, random_state=101, train_size=0.99)

In [5]:
# https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-2-word-vectors
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [6]:
def review_to_wordlist( txt, remove_stopwords=False ):
    txt = BeautifulSoup(txt).get_text()
    txt = re.sub("[^a-zA-Z]"," ", txt)
    words = txt.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)
review_to_wordlist(test['text'].to_list()[1])

['legend',
 'i',
 'saw',
 'one',
 'of',
 'his',
 'paintings',
 'at',
 'moma',
 'in',
 'ny',
 'and',
 'he',
 'actually',
 'put',
 'nuts',
 'bolts',
 'cig',
 'butts',
 'into',
 'the',
 'paint',
 'when',
 'he',
 'mixed',
 'his',
 'paint',
 'for',
 'his',
 'paintings',
 'pretty',
 'frickin',
 'amazing']

In [7]:
train_content = list(map(lambda sentence: review_to_wordlist(sentence), train['text'].to_list()))
train_content[0]

['i',
 'hope',
 'that',
 'one',
 'day',
 'one',
 'of',
 'your',
 'pupils',
 'drinks',
 'sulphuric',
 'acid',
 'that',
 'you',
 'have',
 'carelessly',
 'left',
 'out',
 'in',
 'a',
 'bottle',
 'of',
 'water',
 'it',
 'will',
 'be',
 'a',
 'shame',
 'but',
 'you',
 'will',
 'learn',
 'from',
 'it',
 'if',
 'it',
 'did',
 'happen',
 'hahahahahahahahahaaahahhaha']

In [8]:
from gensim.models import Word2Vec, KeyedVectors

pretrain_w2v = "../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin"
model = KeyedVectors.load_word2vec_format(pretrain_w2v, binary=True)

In [9]:
model.init_sims(replace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """Entry point for launching an IPython kernel.


In [10]:
'hello' in model.key_to_index

True

In [11]:
model.get_vector("hello", norm=True)

array([-0.01640636,  0.00517318, -0.00159814,  0.10050742, -0.07567617,
       -0.00423092, -0.04552395, -0.08040594,  0.00498842,  0.11587914,
       -0.00997684, -0.0294132 , -0.04936688, -0.01345026,  0.00286372,
        0.0558703 ,  0.01101147,  0.05025371,  0.10996693, -0.07744984,
        0.11351426,  0.05202737,  0.0647386 , -0.06030445,  0.03961175,
       -0.02202295, -0.00853574,  0.0351776 ,  0.04641078,  0.02749173,
        0.0205449 , -0.00909001, -0.05114054, -0.06296494, -0.01123318,
       -0.06887714,  0.07981471,  0.00367665,  0.05557469,  0.094004  ,
       -0.03266491, -0.05793957,  0.06532982,  0.03990735, -0.01064196,
        0.05616591, -0.09341277,  0.01448489, -0.03325613,  0.04345468,
       -0.13184208, -0.01145489,  0.03281271,  0.04256785, -0.03207369,
        0.07922349, -0.05173176,  0.11942646,  0.0381337 , -0.08395325,
       -0.08513569,  0.04463712, -0.06355616,  0.00713159,  0.0558703 ,
        0.00134872, -0.08454447, -0.01101147, -0.08868302,  0.05

In [12]:
import numpy as np
indexes = []
def makeFeatureVec(words, num_features=300):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    for word in words:
        if word in model.key_to_index: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model.get_vector(word, norm=True))
    #if nwords < 1:
    #    print(words)
    vectors = np.divide(featureVec, nwords)
    if True in np.isnan(vectors):
        return np.zeros((num_features,),dtype="float32")
    return vectors
makeFeatureVec(['hello', 'love'])

array([ 1.11146960e-02, -2.59782318e-02,  4.05328628e-03,  8.11989307e-02,
       -6.87833056e-02,  1.04274265e-02,  3.21703702e-02, -8.96420777e-02,
       -2.38733161e-02,  9.56597775e-02, -9.90944169e-03, -3.00876573e-02,
       -2.98791230e-02, -1.49649810e-02, -4.26971242e-02,  5.97959086e-02,
        2.96759699e-02,  5.46072125e-02,  5.64025529e-02, -5.18171266e-02,
        4.95243706e-02,  4.09369692e-02,  7.53996372e-02, -5.70690744e-02,
        5.13004214e-02, -1.76033564e-02,  6.07772218e-03,  3.76391038e-02,
        4.41712365e-02, -1.68331396e-02, -1.06933974e-02,  2.16394141e-02,
       -2.26519890e-02, -7.49534555e-03,  2.33144462e-02, -1.99730508e-02,
        8.89802501e-02,  1.80433672e-02,  2.30723172e-02,  1.09624878e-01,
        1.86411403e-02, -6.83379695e-02,  4.59402278e-02,  2.48289220e-02,
       -2.52797343e-02,  8.85663182e-03, -2.36348007e-02,  1.60773993e-02,
       -1.24852499e-02,  3.26680318e-02, -8.65206718e-02,  2.22880505e-02,
       -3.64394858e-03,  

In [13]:
trainDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), train_content))
trainDataVecs[0]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


array([ 1.3525644e-02,  5.7532745e-03,  4.0923253e-02,  5.6992143e-02,
       -3.0962629e-02,  1.0829367e-03,  2.3216564e-02, -4.0458605e-02,
        3.4442082e-02,  4.0593013e-02, -3.2742035e-02, -6.0453892e-02,
       -1.0518829e-02,  4.1925744e-03, -5.6353670e-02,  3.1117527e-02,
        1.8664045e-02,  3.9095819e-02,  2.3144521e-02, -3.3481866e-02,
       -2.6139268e-03,  2.5437463e-02,  3.3107407e-02, -1.1744112e-02,
        2.1203477e-02,  4.6016257e-03, -2.9903157e-02,  1.9701231e-02,
        2.0858124e-02,  8.5473424e-03, -6.7825508e-03,  2.0909684e-02,
       -2.3612477e-02, -2.0251884e-03, -7.4813874e-03, -1.7337275e-05,
        1.3879831e-02, -6.3502025e-03,  1.2756202e-03,  2.7580529e-02,
        3.4236439e-02, -2.7585268e-02,  7.6677859e-02, -1.5877621e-02,
       -8.9541059e-03, -4.0671686e-03, -5.2727666e-03,  9.8666712e-04,
        4.8463456e-03, -6.9256239e-03,  1.4632475e-03,  3.1687483e-02,
        1.0204500e-03, -1.1289590e-02,  7.1920641e-03,  9.8032597e-03,
      

In [14]:
test_content = list(map(lambda sentence: review_to_wordlist(sentence), test['text'].to_list()))
testDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), test_content))

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
forest = GradientBoostingRegressor()
forest = forest.fit(trainDataVecs, train["label"])

In [16]:
# 0.092 ish with norm=True
from sklearn.metrics import explained_variance_score
predictions = forest.predict( testDataVecs )
explained_variance_score(test['label'], predictions)

0.09771658067912092

In [17]:
"""from sklearn.ensemble import RandomForestRegressor
actual_forest = RandomForestRegressor(n_estimators = 100)
actual_forest = actual_forest.fit(trainDataVecs, train["label"])
af_predictions = actual_forest.predict( testDataVecs )
explained_variance_score(test['label'], af_predictions)"""

'from sklearn.ensemble import RandomForestRegressor\nactual_forest = RandomForestRegressor(n_estimators = 100)\nactual_forest = actual_forest.fit(trainDataVecs, train["label"])\naf_predictions = actual_forest.predict( testDataVecs )\nexplained_variance_score(test[\'label\'], af_predictions)'

In [18]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv

comment_id,text
114890,"""
 

Gjalexei, you asked about whether there is an """"anti-editorializing"""" policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.


In [19]:
import csv

texts = []
ids = []
with open('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv', 'r') as rfile:
  rdr = csv.reader(rfile)
  batch = None
  for row in rdr:
    if batch is None:
        batch = True
        continue
    ids.append(row[0])
    texts.append(row[1].replace('"', '').strip())

In [20]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv

comment_id,score
114890,0.5
732895,0.5
1139051,0.5
1434512,0.5


In [21]:
texts[0]

"Gjalexei, you asked about whether there is an anti-editorializing policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.\n\nIf something you write is edited and you're not sure why, please continue to question such edits on the talk page.  Sometimes, you'll learn more about wikipedia policy.  Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"

In [22]:
predict_txts = list(map(lambda sentence: review_to_wordlist(sentence), texts))
predictDataVecs = list(map(lambda sentence: makeFeatureVec(sentence), predict_txts))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [23]:
op = []
id_idx = 0

with open('submission.csv', 'w') as opfile:
  writer = csv.writer(opfile)
  writer.writerow(["comment_id","score"])

  labels = forest.predict( predictDataVecs )
  for label in labels:
      writer.writerow([
        ids[id_idx],
        float(label)
      ])
      id_idx += 1

In [24]:
! head -n 5 submission.csv